In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from QBUS2820 import rmse_jack, r2_jack 
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import RidgeCV
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
from ExtraCode import getResultTable
from sklearn.model_selection import cross_val_score
from QBUS2820 import forward
from sklearn.linear_model import ElasticNet
from QBUS2820 import pcrCV
from QBUS2820 import plsCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
#Stop warnings
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module wil

In [4]:
data = pd.read_csv('Train6_4.csv')
final_train = data.sample(frac=0.6, random_state=1)
final_test = data[data.index.isin(final_train.index)==False]
final_train.head()
y_train = final_train.pop('SalePrice')
y_test = final_test.pop('SalePrice')
y_train = np.log(y_train)



In [5]:
final_train

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleType_WD,SaleType_NonWD,LotFrontage,AgeYearBuilt,AgeYearRemod/Add,AgeGarageYrBlt,Remodeled,GrLivArea,MoSold.1,TotalBsmtSF
8,1296,0,0,1,2,5,3,1005,0,2,...,1,0,70,9,8,9,0,1296,8,1267
791,520,600,0,2,2,5,3,0,0,6,...,1,0,30,5,5,5,1,1200,8,600
571,864,0,0,1,3,5,3,375,239,0,...,1,0,60,40,40,21,1,864,2,864
224,1107,0,0,1,3,5,3,474,38,0,...,1,0,70,43,17,43,0,1107,5,949
498,1187,0,0,1,3,5,3,0,0,6,...,1,0,70,18,17,18,0,1187,10,1168
262,1179,0,0,1,2,5,1,248,0,2,...,1,0,38,22,22,22,1,1179,1,1166
73,1040,0,0,1,2,3,3,0,0,4,...,1,0,60,60,60,60,1,1040,3,0
795,1636,0,0,1,3,5,3,63,0,2,...,1,0,116,4,4,4,1,1636,8,1623
670,926,678,0,1,3,5,3,0,0,6,...,1,0,100,13,12,13,0,1604,9,926
731,1661,0,0,1,3,5,3,0,0,6,...,1,0,48,12,12,12,1,1661,7,1649


In [140]:
pred = []
method = []

### Testing Models

In [141]:
#Forward selection
fwd = forward()
fwd.fit(final_train, y_train)
predforward = fwd.predict(final_test)
method.append('Forward')
pred.append(np.exp(predforward))

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(final_train,y_train)
predFinalBoost = GBoost.predict(final_test)
method.append('GBoost')
pred.append(np.exp(predFinalBoost))

regr = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)
regr = regr.fit(final_train,y_train)
adapred = regr.predict(final_test)
method.append('AdaBoost')
pred.append(np.exp(adapred))

In [142]:
xgdmat = xgb.DMatrix(final_train, y_train) # Create our DMatrix to make XGBoost more efficient
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
testFinaldmat = xgb.DMatrix(final_test)
xgpred = final_gb.predict(testFinaldmat)
method.append('XGBoost')
pred.append(np.exp(xgpred))

In [143]:
pred2 = (np.exp(xgpred)+np.exp(adapred)+np.exp(predFinalBoost))/3
pred.append(pred2)
method.append('XGBoost, ada, GB')

In [144]:
pred3 = (np.exp(xgpred)+np.exp(predFinalBoost))/2

pred5 = (np.exp(xgpred)+np.exp(adapred))/2
pred.append(pred3)
pred.append(pred5)
method.append('xg GB')
method.append('xg adaboost')

In [145]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(final_train,y_train)

predFinalExtRandomForestlad = regr2.predict(final_test)
pred.append(np.exp(predFinalExtRandomForestlad))
method.append('Random Forest')

In [146]:
#LASSO
lasso = Lasso(alpha = 1)
lasso.fit(final_train, np.ravel(y_train)) 
pred_L = lasso.predict(final_test)
method.append('LASSO')
pred.append(np.exp(pred_L))

In [147]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(final_train, y_train)
y_pred_rg = (rg.predict(final_test))
y_pred_rg

pred.append(np.exp(y_pred_rg))
method.append('Ridge alpha 10')

In [148]:
regr = ElasticNet(random_state=0, alpha = 1)
    
regr.fit(final_train, y_train)    
pred13 = regr.predict(final_test)
pred.append(np.exp(pred13))
method.append('ENET')

In [149]:
pred12 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(predFinalExtRandomForestlad)+np.exp(pred13))/4
pred.append(pred12)
method.append('XGBoost, GBoost Random Forest ENET')

In [150]:
pred13 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg))/3
pred.append(pred13)
method.append('XGBoost, GBoost Ridge')

In [151]:
pred14 = (np.exp(xgpred)+np.exp(predFinalBoost)+np.exp(y_pred_rg)+np.exp(predforward))/4
pred.append(pred14)
method.append('XGBoost, GBoost Ridge Forward')

In [152]:
pred15 = (np.exp(xgpred)+np.exp(predFinalBoost)+ np.exp(predFinalExtRandomForestlad))/3
pred.append(pred15)
method.append('XGBoost, GBoost, RandomTree')

In [153]:

pred15 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred15)
method.append('XGBoost, GBoost Ridge Forward 30 30 20 20')

In [154]:
pred16 = ((0.4*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred16)
method.append('XGBoost, GBoost Ridge Forward 40 40 10 10')

In [155]:
pred17 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred17)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [156]:
pred18 = ((0.35*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.15*np.exp(y_pred_rg))+(0.15*np.exp(predforward)))
pred.append(pred18)
method.append('XGBoost, GBoost Ridge Forward 35 35 15 15')

In [157]:
pred19 = ((0.25*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred19)
method.append('XGBoost, GBoost Ridge Forward 25 35 20 20')

In [158]:
pred20 = ((0.2*np.exp(xgpred))+(0.4*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred20)
method.append('XGBoost, GBoost Ridge Forward 20 40 20 20')

In [159]:
pred21 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred21)
method.append('XGBoost, GBoost Ridge Forward 20 30 20 30')

In [160]:
pred22 = ((0.2*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred22)
method.append('XGBoost, GBoost Ridge Forward 20 30 30 20')

In [161]:
pred23 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred23)
method.append('XGBoost, GBoost Ridge Forward 20 20 30 30')

In [162]:
pred24 = ((0.3*np.exp(xgpred))+(0.5*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 30 50 10 10')

In [163]:
pred24 = ((0.2*np.exp(xgpred))+(0.2*np.exp(predFinalBoost))+(0.2*np.exp(y_pred_rg))+(0.4*np.exp(predforward)))
pred.append(pred24)
method.append('XGBoost, GBoost Ridge Forward 20 20 20 40')

In [164]:
pred25 = ((0.3*np.exp(xgpred))+(0.3*np.exp(predFinalBoost))+(0.3*np.exp(y_pred_rg))+(0.1*np.exp(predforward)))
pred.append(pred25)
method.append('XGBoost, GBoost Ridge Forward 30 30 30 10')

In [165]:
pred26 = ((0.26*np.exp(xgpred))+(0.27*np.exp(predFinalBoost))+(0.27*np.exp(y_pred_rg))+(0.2*np.exp(predforward)))
pred.append(pred26)
method.append('XGBoost, GBoost Ridge Forward 26 27 27 20')

In [166]:
pred27 = ((0.23*np.exp(xgpred))+(0.24*np.exp(predFinalBoost))+(0.24*np.exp(y_pred_rg))+(0.3*np.exp(predforward)))
pred.append(pred27)
method.append('XGBoost, GBoost Ridge Forward 23 24 24 30')

In [167]:
pred21 = ((0.15*np.exp(xgpred))+(0.35*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.35*np.exp(predforward))+ (0.05*np.exp(predFinalExtRandomForestlad)))
pred.append(pred21)
method.append('XGBoost, GBoost Ridge Forward RandomForest 15 35 10 35 5')

In [168]:
getResultTable(method,pred)

,Test RMSE,SE,Jack R2,SE,MAE,R-square
Forward,24192.077,6076.362,0.880,0.056,13387.449,0.880
GBoost,19461.466,2971.564,0.923,0.016,13173.110,0.923
AdaBoost,25556.193,3670.700,0.867,0.024,17610.516,0.867
XGBoost,20698.479,2773.130,0.912,0.015,13922.274,0.912
"XGBoost, ada, GB",20275.404,3393.425,0.916,0.019,13525.257,0.916
xg GB,19498.850,2954.158,0.922,0.016,12998.155,0.922
xg adaboost,21520.878,3455.066,0.905,0.020,14532.348,0.905
Random Forest,26155.966,3511.307,0.860,0.023,18106.706,0.860
LASSO,34657.284,7341.991,0.755,0.097,20088.100,0.755
Ridge alpha 10,22308.338,6212.873,0.898,0.052,12614.385,0.898


In [41]:
def getResultTable(rows, predictions):
    columns=['Test RMSE', 'SE', 'Jack R2', 'SE', 'MAE', 'R-square']
    results=pd.DataFrame(0.0, columns=columns, index=rows)

    for row,pred in zip(range(0,len(rows)),predictions):
        results.iloc[row,0], results.iloc[row,1] = rmse_jack(y_test, pred)
        results.iloc[row,2], results.iloc[row,3] = (r2_jack(y_test, pred))
        results.iloc[row,4] = mean_absolute_error(y_test, pred)
        results.iloc[row,5] = r2_score(y_test,pred)
    return results.round(3)

## Kaggle Predictions

In [169]:
data = pd.read_csv('Train6_4.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_4.csv')
y_price = np.log(y_price)
xgdmat = xgb.DMatrix(data, y_price) # Create our DMatrix to make XGBoost more efficient
testFinaldmat = xgb.DMatrix(kaggle)

In [170]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':3, 'min_child_weight':1} 

final_gb = xgb.train(our_params, xgdmat, num_boost_round = 432)
predictionxg = final_gb.predict(testFinaldmat)

In [171]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
GBoost.fit(data,y_price)
predFinalBoost = GBoost.predict(kaggle)


In [172]:
alpha = 10
rg = Ridge(alpha = alpha)
rg.fit(data, y_price)
y_pred_rg = (rg.predict(kaggle))

In [173]:
#Forward selection
fwd = forward()
fwd.fit(data, y_price)
predforward = fwd.predict(kaggle)

In [174]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr2.fit(data,y_price)
predFinalExtRandom = regr2.predict(kaggle)
 

In [186]:
finalprediction = ((0.15*np.exp(predictionxg))+(0.35*np.exp(predFinalBoost))+(0.1*np.exp(y_pred_rg))+(0.35*np.exp(predforward))+ (0.05*np.exp(predFinalExtRandom)))

In [187]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction.to_csv("Day24_1.csv", index=False)

In [188]:
prediction

,Id,Prediction
0,1,95025.308665
1,2,165410.069582
2,3,138890.057752
3,4,188513.590716
4,5,145409.519894
5,6,344558.483289
6,7,132798.324040
7,8,289360.199806
8,9,117386.066938
9,10,127361.076481


In [191]:
from scipy.stats import norm, skew

numeric_feats = final_train.dtypes[final_train.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = final_train[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness

#Looking at skewness


Skew in numerical features: 



,Skew
Exterior1st_ImStucc,21.886116
Condition1_PosA,21.886116
Condition1_RRNn,21.886116
Exterior1st_BrkComm,21.886116
PoolArea,21.886116
MiscVal,15.719250
Condition1_RRNe,15.427384
MasVnrType_BrkCmn,12.556788
LotConfig_FR3,12.556788
Neighborhood_Blmngtn,12.556788


In [197]:
#Validation function
n_folds = 5

def mae_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(final_train)
    mae= -cross_val_score(model, final_train.values, y_train, scoring="neg_mean_absolute_error", cv = kf)
    return(mae)

In [201]:
def mae(y, y_pred):
    return (mean_absolute_error(y, y_pred))

In [212]:
#Base models
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.kernel_ridge import KernelRidge

LASSO = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

ExtraTrees = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)

GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

Adaboost = AdaBoostRegressor(loss='linear', learning_rate = 1, n_estimators = 350)

model_xgb1 = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             nthread = -1)
#Kernel Ridge Regression
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=1, coef0=2.5)

#Ridge Regression
RIDGE = make_pipeline(RobustScaler(), Ridge(alpha=0.1, random_state = 1))

#ENET
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [203]:

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,GridSearchCV
score = mae_cv(LASSO)
print("LASSO SCORE {}".format(score.mean()))
LASSO.fit(final_train,y_train)
LASSOpred = LASSO.predict(final_test)
print(mae(y_test,np.exp(LASSOpred)))

LASSO SCORE 0.0697885348122233
12466.2097302


In [204]:
score = mae_cv(ExtraTrees)
print("Extra Trees SCORE {}".format(score.mean()))
ExtraTrees.fit(final_train,y_train)
pred = ExtraTrees.predict(final_test)
print(mae(y_test,np.exp(pred)))

Extra Trees SCORE 0.08924277643647724
16916.1578301


In [205]:
score = mae_cv(GBoost)
print("Gradient Boost SCORE {}".format(score.mean()))
GBoost.fit(final_train,y_train)
pred = GBoost.predict(final_test)
print(mae(y_test,np.exp(pred)))

Gradient Boost SCORE 0.079069862674465
13173.1097931


In [206]:
score = mae_cv(Adaboost)
print("Adaboost Boost SCORE {}".format(score.mean()))
Adaboost.fit(final_train,y_train)
pred = Adaboost.predict(final_test)
print(mae(y_test,np.exp(pred)))

Adaboost Boost SCORE 0.10584078798404672
18073.9744487


In [207]:
score = mae_cv(model_xgb1)
print("XGBoost1 SCORE {}".format(score.mean()))
model_xgb1.fit(final_train,y_train)
pred = model_xgb1.predict(final_test)
print(mae(y_test,np.exp(pred)))

XGBoost1 SCORE 0.08189873981811885
14486.8217804


In [208]:
score = mae_cv(KRR)
print("Kernel Ridge Regression SCORE {}".format(score.mean()))
KRR.fit(final_train,y_train)
pred = KRR.predict(final_test)
print(mae(y_test,np.exp(pred)))

Kernel Ridge Regression SCORE 0.07560826129354085
12996.7647127


In [209]:
score = mae_cv(ENet)
print("Elastic Net Regression SCORE {}".format(score.mean()))
ENet.fit(final_train,y_train)
pred = ENet.predict(final_test)
print(mae(y_test,np.exp(pred)))

Elastic Net Ridge Regression SCORE 0.06973206304107792
12467.0714515


In [213]:
score = mae_cv(RIDGE)
print("Ridge SCORE {}".format(score.mean()))
RIDGE.fit(final_train,y_train)
Ridgepred = RIDGE.predict(final_test)
print(mae(y_test,np.exp(Ridgepred)))

Ridge SCORE 0.07481164795667097
13475.8732859


In [215]:
#Taking average of models
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)  

In [216]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [217]:
averaged_models = AveragingModels(models = (Adaboost, GBoost, model_xgb1, LASSO))
score = mae_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.0753 (0.0044)



In [218]:
averaged_models = AveragingModels(models = (Adaboost, GBoost, model_xgb1, LASSO,ENet,KRR,ExtraTrees))
score = mae_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.0716 (0.0059)



In [219]:
averaged_models = AveragingModels(models = (GBoost, LASSO,ENet,KRR))
score = mae_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.0681 (0.0048)



In [220]:
averaged_models = AveragingModels(models = (GBoost,ENet,KRR))
score = mae_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

 Averaged base models score: 0.0688 (0.0051)



In [221]:
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
#Python class used for stacking models
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=15)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X.values[train_index], y.values[train_index])
                y_pred = instance.predict(X.values[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [222]:
stacked_averaged_modelsa = StackingAveragedModels(base_models = (ENet, GBoost,KRR),
                                                 meta_model = LASSO)

stacked_averaged_modelsa.fit(final_train,y_train)
stacked_train_pred2 = stacked_averaged_modelsa.predict(final_test.values)
print(mae(y_test,np.exp(stacked_train_pred2)))

#Best model

12033.4052933


In [223]:
stacked_averaged_models3 = StackingAveragedModels(base_models = (GBoost, LASSO,ENet,KRR),
                                                 meta_model = LASSO)

stacked_averaged_models3.fit(final_train,y_train)
stacked_train_pred3 = stacked_averaged_models3.predict(final_test.values)
print(mae(y_test,np.exp(stacked_train_pred3)))

12040.048442


In [224]:
averaged_models = AveragingModels(models = (GBoost,ENet,KRR))
averaged_models.fit(final_train,y_train)
predaverage = averaged_models.predict(final_test)
print(mae(y_test,np.exp(predaverage)))

#use this one too

11922.3074546


In [225]:
averaged_models = AveragingModels(models = (GBoost, LASSO,ENet,KRR))
averaged_models.fit(final_train,y_train)
predaverage = averaged_models.predict(final_test)
print(mae(y_test,np.exp(predaverage)))

11940.3400886


In [226]:
data = pd.read_csv('Train6_4.csv')
y_price = data.pop('SalePrice')
kaggle = pd.read_csv('Test6_4.csv')
y_price = np.log(y_price)

In [227]:
averaged_models = AveragingModels(models = (GBoost,ENet,KRR))
averaged_models.fit(data,y_price)
predaverage = averaged_models.predict(kaggle)


In [228]:
stacked_averaged_models = StackingAveragedModels(base_models = ( GBoost,ENet,KRR),
                                                 meta_model = LASSO)
stacked_averaged_models.fit(data,y_price)
stacked_train_pred = stacked_averaged_models.predict(kaggle.values)

In [229]:
finalprediction = np.exp((0.7*predaverage)+ (0.3*stacked_train_pred))

In [238]:
finalprediction100 = np.exp((0.9*predaverage)+ (0.1*stacked_train_pred))
finalprediction100

array([  96742.00257547,  158476.43685299,  136963.12873598, ...,
         86486.02709783,  192353.07824109,  217972.92663946])

In [230]:
finalprediction

array([  96266.89348615,  159274.96824086,  136890.73664251, ...,
         86119.27915541,  192676.72305851,  217911.45084091])

In [232]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction2 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction})
#Saving results into CSV file 
prediction2.to_csv("Day24_2.csv", index=False)

In [233]:
prediction2

,Id,Prediction
0,1,96266.893486
1,2,159274.968241
2,3,136890.736643
3,4,188846.053433
4,5,142958.896423
5,6,367177.840598
6,7,132387.612329
7,8,293156.737052
8,9,114962.271906
9,10,125709.020401


In [111]:
finalprediction3 = (0.15*np.exp(predictionxg))+(0.05*np.exp(predFinalBoost))+(0.4*np.exp(y_pred_rg))+(0.4*np.exp(predforward))

In [112]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction3 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction3})
#Saving results into CSV file 
prediction3.to_csv("Day23_3.csv", index=False)

In [113]:
prediction3

,Id,Prediction
0,1,98061.376273
1,2,161003.973830
2,3,140388.377990
3,4,187518.053981
4,5,142757.058239
5,6,372343.905353
6,7,128656.124102
7,8,294450.884408
8,9,115424.936014
9,10,132832.747465


In [124]:
#LASSO
lasso = Lasso(alpha = 1)
lasso.fit(data, np.ravel(y_price)) 
pred_L_final = lasso.predict(kaggle)

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [140]:
np.exp(pred_L_final)

array([ 117625.3068124 ,  154406.58809745,  135270.46006291, ...,
        106935.400622  ,  164524.5301393 ,  224045.45467818])

In [125]:
#Extremely Random forest
regr2 = ExtraTreesRegressor(criterion='mae',max_depth=None,min_samples_split=2)
regr2 = regr.fit(data,y_price)

predFinalExtRandomForestlad = regr2.predict(kaggle)

/Users/ChristopherHyland/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [141]:
finalprediction4 = ((np.exp(predFinalExtRandomForestlad)*(1/5))+(np.exp(prediction)*(1/5))+(np.exp(predFinalBoost)*(1/5))+(np.exp(y_pred_rg)*(1/5)) + (np.exp(predforward)*(1/5)))

In [142]:
finalprediction4

array([  97623.67532531,  159917.41864414,  133822.27050019, ...,
         92461.51084685,  189523.84021274,  215180.31456205])

In [143]:
#This is for the indices
ind = np.arange(1,1609)
headers = ['Id','Prediction']
prediction4 = pd.DataFrame({'Id':ind, 'Prediction':finalprediction4})
#Saving results into CSV file 
prediction4.to_csv("Day18_4.csv", index=False)